In [ ]:
#Delete sqlite file if necesary to start fresh
!rm hawaii.sqlite

In [1]:
#import dependencies
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
#Create base ORM (Object Relational Mapping) class
Base = declarative_base()

In [3]:
#read cleaned data (no missing values)
stations_df = pd.read_csv(os.path.join('resources','clean_hawaii_stations.csv'))
meas_df = pd.read_csv(os.path.join('resources','clean_hawaii_measurements.csv'))
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [4]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [6]:
class Station(Base):
    __tablename__ = 'station'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, station_id={self.station}, name={self.name}, \
        latitude={self.latitude}, longitude={self.longitude}, elevation={self.elevation}"

In [7]:
class Measurement(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)
   
    def __repr__(self):
        return f"id={self.id}, station_id={self.station}, date={self.date}, prcp={self.prcp}, tobs={self.tobs}"

In [8]:
#create all the tables in the databse
Base.metadata.create_all(engine)

In [9]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
stations_data = stations_df.to_dict(orient='records')
measurements_data = meas_df.to_dict(orient='records')

In [11]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [12]:
# Save the reference to the tables`
station_table = sqlalchemy.Table('station', metadata, autoload=True)
measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)

In [13]:
# Use `table.delete()` to remove any pre-existing data.
conn.execute(station_table.delete())
conn.execute(measurement_table.delete())

In [14]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(station_table.insert(), stations_data)
conn.execute(measurement_table.insert(), measurements_data)

In [15]:
# Confirm data is loaded by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [16]:
# Confirm data is loaded by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73)]